In [1]:
import sympy as sp
import numpy as np

sp.init_printing(use_latex=True)


In [5]:
L1_length = 0.110
L2_length = 0.311
L3_length = 0.273

#q1, q2, q3 = sp.symbols('q_1 q_2 q_3', real=True)
x, y, z, yaw, pitch, roll, q1, q2, q3 = sp.symbols('x_b y_b z_b yaw pitch roll q_1 q_2 q_3', real=True)

# Parameters
L1, L2, L3 = sp.symbols('L_1 L_2 L_3', real=True, positive=True)

In [11]:
# ROTATIONS
# Intrinsic ZYX
Rz = sp.Matrix([
    [sp.cos(yaw), -sp.sin(yaw), 0],
    [sp.sin(yaw),  sp.cos(yaw), 0],
    [0,            0,           1]
])

Ry = sp.Matrix([
    [sp.cos(pitch),  0, sp.sin(pitch)],
    [0,              1, 0            ],
    [-sp.sin(pitch), 0, sp.cos(pitch)]
])

Rx = sp.Matrix([
    [1, 0,            0           ],
    [0, sp.cos(roll), -sp.sin(roll)],
    [0, sp.sin(roll),  sp.cos(roll)]
])

# Intrinsic ZYX: Postmultiplication
R_zyx_intrinsic = Rz * Ry * Rx

# Extrinsic XYZ: Premultiplication
R_xyz_extrinsic = Rz * Ry * Rx

sp.pprint(R_xyz_extrinsic)
sp.pprint(R_zyx_intrinsic)

⎡cos(pitch)⋅cos(yaw)  sin(pitch)⋅sin(roll)⋅cos(yaw) - sin(yaw)⋅cos(roll)  sin( ↪
⎢                                                                              ↪
⎢sin(yaw)⋅cos(pitch)  sin(pitch)⋅sin(roll)⋅sin(yaw) + cos(roll)⋅cos(yaw)  sin( ↪
⎢                                                                              ↪
⎣    -sin(pitch)                     sin(roll)⋅cos(pitch)                      ↪

↪ pitch)⋅cos(roll)⋅cos(yaw) + sin(roll)⋅sin(yaw)⎤
↪                                               ⎥
↪ pitch)⋅sin(yaw)⋅cos(roll) - sin(roll)⋅cos(yaw)⎥
↪                                               ⎥
↪            cos(pitch)⋅cos(roll)               ⎦
⎡cos(pitch)⋅cos(yaw)  sin(pitch)⋅sin(roll)⋅cos(yaw) - sin(yaw)⋅cos(roll)  sin( ↪
⎢                                                                              ↪
⎢sin(yaw)⋅cos(pitch)  sin(pitch)⋅sin(roll)⋅sin(yaw) + cos(roll)⋅cos(yaw)  sin( ↪
⎢                                                                              ↪
⎣    -sin(pitch)    

In [6]:
T_Ib = sp.Matrix([[sp.cos(yaw)*sp.cos(pitch), sp.cos(yaw)*sp.sin(pitch)*sp.sin(roll)-sp.sin(yaw)*sp.cos(roll), sp.cos(yaw)*sp.sin(pitch)*sp.cos(roll)+sp.sin(yaw)*sp.sin(roll), x],
                  [sp.sin(yaw)*sp.cos(pitch), sp.sin(yaw)*sp.sin(pitch)*sp.sin(roll)+sp.cos(yaw)*sp.cos(roll), sp.sin(yaw)*sp.sin(pitch)*sp.cos(roll)-sp.cos(yaw)*sp.sin(roll), y],
                  [-sp.sin(pitch), sp.cos(pitch)*sp.sin(roll), sp.cos(pitch)*sp.cos(roll), z],
                  [0, 0, 0, 1]])

T_b0 = sp.Matrix([[0, 0, 1, 0],
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 0, 1]])

T_01 = sp.Matrix([[sp.cos(q1), -sp.sin(q1), 0, 0],
                [sp.sin(q1),  sp.cos(q1), 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

T_12 = sp.Matrix([[-sp.cos(q2), sp.sin(q2), 0, -L1],
                [0, 0, 1, 0],
                [sp.sin(q2), sp.cos(q2), 0, 0],
                [0, 0, 0, 1]])

T_23 = sp.Matrix([[-sp.cos(q3), sp.sin(q3), 0, L2],
                [0, 0, 1, 0],
                [sp.sin(q3), sp.cos(q3), 0, 0],
                [0, 0, 0, 1]])

T_3e = sp.Matrix([[0, 0, -1, -L3],
                [0, 1, 0, 0],
                [1, 0, 0, 0],
                [0, 0, 0, 1]])

T_be = sp.trigsimp(sp.expand(T_b0*T_01*T_12*T_23*T_3e))
    
# Set parameters
T_be = T_be.subs([(L1, L1_length), (L2, L2_length), (L3, L3_length)])

In [9]:
# Forward kinematics function
FK = sp.Matrix([T_be[0,3],
                T_be[1,3],
                T_be[2,3],
                sp.atan2(T_be[2,1], T_be[2,2]),
                sp.asin(T_be[0,3]),
                sp.atan2(T_be[1,0], T_be[0,0])])
sp.pprint(FK)

sp.pprint("State dependencies")
sp.pprint(f"xe = f({FK[0,0].free_symbols})")
print(f"ye = f({FK[1,0].free_symbols})")
print(f"ze = f({FK[2,0].free_symbols})")
print(f"yawe = f({FK[3,0].free_symbols})")
print(f"rolle = f({FK[4,0].free_symbols})")
print(f"pitche = f({FK[5,0].free_symbols})")



⎡                               (0.273⋅cos(q₃) + 0.311)⋅sin(q₂)                ↪
⎢                                                                              ↪
⎢0.273⋅sin(q₁)⋅sin(q₃) - 0.273⋅cos(q₁)⋅cos(q₂)⋅cos(q₃) - 0.311⋅cos(q₁)⋅cos(q₂) ↪
⎢                                                                              ↪
⎢-0.273⋅sin(q₁)⋅cos(q₂)⋅cos(q₃) - 0.311⋅sin(q₁)⋅cos(q₂) - 0.11⋅sin(q₁) - 0.273 ↪
⎢                                                                              ↪
⎢atan2(-sin(q₁)⋅sin(q₃)⋅cos(q₂) + cos(q₁)⋅cos(q₃), -sin(q₁)⋅cos(q₂)⋅cos(q₃) -  ↪
⎢                                                                              ↪
⎢                            asin((0.273⋅cos(q₃) + 0.311)⋅sin(q₂))             ↪
⎢                                                                              ↪
⎣                               atan2(sin(q₂)⋅cos(q₁), cos(q₂))                ↪

↪                 ⎤
↪                 ⎥
↪  - 0.11⋅cos(q₁) ⎥
↪                 ⎥
↪ ⋅sin(q₃)⋅cos(q₁)⎥
↪       